In [1]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('data/ds_pic/gen2_revised.png')

driving_video = imageio.mimread('data/kon_ep1_mid_175.mp4')

#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, orig=None, generated=None, compared=None, ep=None, clip=None, save=True):
    fig = plt.figure(figsize=(8 + 4 * (orig is not None) + 4 * (generated is not None) + 4 * (compared is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        title = 'Source image, Kon{0}_{1}'.format(ep, clip)
        if orig is not None:
            cols.append(orig[i])
            title += ', celeb chkpt'
        if generated is not None:
            cols.append(generated[i])
            title += ', kon1 chkpt'
        if compared is not None:
            cols.append(compared[i])
            title += ', kon1_5 chkpt'
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')        
        plt.title(title)
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    if save:
        ani.save('kon{0}_{1}.mp4'.format(ep, clip))
    plt.close()
    return ani


Bad key text.latex.unicode in file /home/ray/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /home/ray/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /home/ray/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplo

In [2]:
from Fomm.demo import load_checkpoints
from Fomm.demo import make_animation
from skimage import img_as_ubyte

celeb_gen, celeb_kp_det = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='data/vox-cpk.pth.tar')

gen_k1, kp_det_k1 = load_checkpoints(config_path='config/kon_ep1_ray_v1.yaml', 
                            checkpoint_path='log/kon_ep1_ray_v1 01_10_20_19.34.05/00000149-checkpoint.pth.tar')


gen_k1_5, kp_det_k1_5 = load_checkpoints(config_path='config/kon_ep1_ray_v1.yaml', 
                            checkpoint_path='log/kon1-5_sq_crop 05_10_20_21.07.58/00000049-checkpoint.pth.tar')

In [ ]:
driving_video = imageio.mimread(
    'kon_ep1_mid_175.mp4')
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

orig_pred = make_animation(source_image, driving_video, celeb_gen, celeb_kp_det, relative=True)
pred_k1 = make_animation(source_image, driving_video, gen_k1, kp_det_k1, relative=True)
pred_k1_5 = make_animation(source_image, driving_video, gen_k1_5, kp_det_k1_5, relative=True)\

In [ ]:
HTML(display(source_image, driving_video, orig_pred, pred_k1,
             pred_k1_5, ep=1, clip=175).to_html5_video())

In [9]:
## list of vids to animate for

clip_list = [155, 10, 100, 102, 104, 110, 112, 113, 114, 119, 12, 13, 131, 132, 133, 136, 14, 140, 143, 144,
             145, 147, 149, 15, 150, 154, 156, 159, 16, 160, 161, 163, 164, 167, 17, 170, 171, 174, 175, 176,
             179, 183, 185, 186, 19, 190, 191, 192, 193, 194, 198, 199, 21, 23, 28, 29, 3, 31, 36, 37, 39,
            44, 45, 47, 51, 52, 53, 54, 55, 56, 59, 6, 60, 61, 65, 66, 67, 7, 72, 74, 75, 77, 79, 8, 80, 81,
            82, 83, 84, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97]

In [6]:
def get_save_result(clip_num, ep_num):
    driving_video = imageio.mimread(
    'kon_{0}/clips/kon{0}_traj{1}.mp4'.format(ep_num, clip_num))
    driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

    orig_pred = make_animation(source_image, driving_video, celeb_gen, celeb_kp_det, relative=True)
    pred_k1 = make_animation(source_image, driving_video, gen_k1, kp_det_k1, relative=True)
    pred_k1_5 = make_animation(source_image, driving_video, gen_k1_5, kp_det_k1_5, relative=True)

    HTML(display(source_image, driving_video, orig_pred, pred_k1,
                 pred_k1_5, ep=ep_num, clip=clip_num).to_html5_video())

In [ ]:
for i in clip_list:
    get_save_result(i, 2)

100%|██████████| 161/161 [00:04<00:00, 39.87it/s]
